#  This benchmark does relative CPU comparisons.

It starts by grouping all jobs per CPU model and configuration and taskID. For each of these two values are calculated: average CPU time per event and sum of the number of events processed. 

We calculate relative performance between all the CPU modes/configurations based on all taskIDs.

We optimizite placement of different CPUs on the "power scale" that goes between 0 and 1, by making multidimensional optimization trying to minimize errors in relative ratios for all the combinations of CPU pairs.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Get datasets

In [2]:
jobs = pd.DataFrame()
input_files=['job_node_info.csv']
tmp=[]
for input_file in input_files:
    tmp.append(pd.read_csv(input_file))
jobs = pd.concat(tmp)

jobs = jobs[jobs['nevents']>20]
del jobs['cputime']
del jobs['walltime']
del jobs['cpueff']
del jobs['wallPerEvent']
del jobs['inputsize']
del jobs['processingtype']

jobs.head()

,host,nevents,cpuPerEvent,taskid
0,ba-3-9-17.cr.cnaf.infn.it,50,2880.080000,11330815
1,td113.pic.es,250,85.896000,11470165
2,b63ab874b9.cern.ch,1000,45.890000,11374294
3,t2-wn-91.mi.infn.it,3878,0.642599,11570096
5,b6bd945e9b.cern.ch,230,21.091304,11449385


In [3]:
input_files=['benchmark_node_info.csv']
tmp=[]
for input_file in input_files:
    tmp.append(pd.read_csv(input_file))
benchmarks = pd.concat(tmp)

print(benchmarks.columns)

del benchmarks['ip']
del benchmarks['site']
del benchmarks['meminfo']
#print(benchmarks.columns)

benchmarks.head()

Index(['ip', 'host', 'site', 'meminfo', 'mpnum', 'cpuname', 'VM', 'cpunum',
       'coresPerSocket', 'threadsPerCore'],
      dtype='object')


,host,mpnum,cpuname,VM,cpunum,coresPerSocket,threadsPerCore
0,r25-n14.ph.liv.ac.uk,1,Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz,False,24,6,2
1,cern-atlas-dcb8d293-709a-42f5-b891-8f3eface243...,8,Intel(R) Xeon(R) CPU E5-2650 v2 @ 2.60GHz,True,8,1,1
2,wn-200-03-27-01-a.cr.cnaf.infn.it,8,AMD Opteron(tm) Processor 6320,False,16,4,2
3,wg60.grid.hep.ph.ic.ac.uk,8,Intel(R) Xeon(R) CPU X5650 @ 2.67GHz,False,24,6,2
4,vac037-03.beowulf.cluster,1,QEMU Virtual CPU version 1.5.3,True,8,1,1


#### make new CPU name that adds "VM" and "HT" if needed

In [4]:
def getCPU(cpu,vm,tpc):
    res=cpu.strip()
    if tpc>1: res+=" HT"
    if vm: res+=" VM"
    return res
benchmarks['CPU'] = benchmarks.apply(lambda x: getCPU(x['cpuname'], x['VM'], x['threadsPerCore']), axis=1)
del benchmarks['cpuname']
del benchmarks['VM']
del benchmarks['coresPerSocket']
del benchmarks['threadsPerCore']
del benchmarks['mpnum']
del benchmarks['cpunum']
benchmarks=benchmarks.set_index('host')
benchmarks.head()

,CPU
host,
r25-n14.ph.liv.ac.uk,Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz HT
cern-atlas-dcb8d293-709a-42f5-b891-8f3eface243c.cern.ch,Intel(R) Xeon(R) CPU E5-2650 v2 @ 2.60GHz VM
wn-200-03-27-01-a.cr.cnaf.infn.it,AMD Opteron(tm) Processor 6320 HT
wg60.grid.hep.ph.ic.ac.uk,Intel(R) Xeon(R) CPU X5650 @ 2.67GH...
vac037-03.beowulf.cluster,QEMU Virtual CPU version 1.5.3 VM


### join dataframes and filter out unneded rows

In [5]:
jobs['CPU']=(jobs['host']).map(benchmarks['CPU'])
del jobs['host']

#jobs.CPU=jobs.processingtype
jobs.head()

# benchmarks = benchmarks[benchmarks.cpuPerMB > 0]
# benchmarks = benchmarks[benchmarks.cpuPerEvent > 0]

,nevents,cpuPerEvent,taskid,CPU
0,50,2880.080000,11330815,AMD Opteron(tm) Processor 6376 HT
1,250,85.896000,11470165,Intel(R) Xeon(R) CPU E5-2640 v3 @ 2.60GHz HT
2,1000,45.890000,11374294,Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz VM
3,3878,0.642599,11570096,AMD Opteron(tm) Processor 6320 HT
5,230,21.091304,11449385,Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz VM


#### sum-up per taskid and CPU

In [6]:
print(jobs.shape)
raw = jobs.groupby(["CPU","taskid"]).agg({'nevents': ['sum','count'], 'cpuPerEvent':['mean']})
print(raw.shape)
raw.head(20)



(9797514, 4)
(263949, 3)


cpuPerEvent nevents      
                                                        mean     sum count
CPU                                     taskid                            
AMD FX(tm)-8150 Eight-Core Processor HT 11173045  771.706667     150     3
                                        11173064  956.730000     300     6
                                        11173073   68.423929    2800    14
                                        11173076  206.093247    7700    77
                                        11173080  289.194000     500     5
                                        11173083  867.895152    3300    66
                                        11173085   90.455400    5000    10
                                        11173093  573.474531    6400    64
                                        11173100  506.042500     400     4
                                        11173111   65.839429    3500     7
                                        11173115  106.393333    4800    24
                                        11173117   43.087632   19000    19
                                        11173119  161.342000    4000    20
                                        11173121  247.725000    1600     8
                                        11174129   10.616000    2000     1
                                        11330032  219.445333    4500     9
                                        11330036  128.818435   11500    23
                                        11330050   50.712800    2500     5
                                        11330053   99.649857   14000    14
                                        11330803  450.670000     200     2

### find unique CPUs and unique tasks split in dataframes based on cpuname

In [7]:
unique_cpus = raw.index.levels[0].tolist()
unique_taskids = raw.index.levels[1].tolist()
print("CPUs:",len(unique_cpus),"\tTaskIDs:",len(unique_taskids))

CPUs: 170 	TaskIDs: 29141


In [8]:
CPU_types = {elem : pd.DataFrame for elem in unique_cpus}

to_remove=[]
#filling up data frames
for key in CPU_types.keys():
    CPU_types[key] = raw.iloc[raw.index.get_level_values('CPU') == key]
    CPU_types[key].index = CPU_types[key].index.droplevel(0)
    jobs_done_by_this_cpu = CPU_types[key]['nevents']['count'].sum()
    if jobs_done_by_this_cpu<2000: to_remove.append(key)

#to_remove.append('Intel Core Processor (Haswell, no TSX) VM')

for c in to_remove:
    print('removing:', c)
    unique_cpus.remove(c)
    del CPU_types[c]

#test_df = CPU_types['AMD FX(tm)-8150 Eight-Core Processor HT']
#test_df.head()

removing: Intel(R) Xeon(R) CPU E5-2470 v2 @ 2.40GHz HT
removing: QEMU Virtual CPU version 1.1.2 VM
removing: Intel(R) Core(TM) i7-3770 CPU @ 3.40GHz HT
removing: Intel(R) Xeon(R) CPU           E5504  @ 2.00GHz
removing: Intel Xeon E312xx (Sandy Bridge) VM
removing: Intel(R) Core(TM) i7 CPU         870  @ 2.93GHz HT
removing: Intel(R) Xeon(R) CPU           E5649  @ 2.53GHz
removing: AMD Opteron(tm) Processor 6176
removing: AMD Opteron(tm) Processor 4226 HT
removing: Intel(R) Xeon(R) CPU           E5335  @ 2.00GHz
removing: Intel(R) Xeon(R) CPU E5-2695 v3 @ 2.30GHz HT
removing: Intel Celeron_4x0 (Conroe/Merom Class Core 2) VM
removing: Intel(R) Xeon(R) CPU E5-2670 0 @ 2.60GHz
removing: Intel(R) Xeon(R) CPU           X5647  @ 2.93GHz HT
removing: AMD Opteron(tm) Processor 6128
removing: Intel(R) Core(TM)2 Quad CPU    Q8300  @ 2.50GHz
removing: Intel Core i7 9xx (Nehalem Class Core i7) VM
removing: Westmere E56xx/L56xx/X56xx (Nehalem-C) VM
removing: AMD FX(tm)-8350 Eight-Core Processor HT


### define function to sum up data for two CPUs

In [9]:
def sumup(c1, c2):
    # print(CPU_types[c1])
    # print(CPU_types[c2])
    #inner join in oder to remove tasks that did not have both of these CPUs
    mer=CPU_types[c1].join(CPU_types[c2], lsuffix='_f', rsuffix='_s', how='inner')
    
    if mer.shape[0] == 0: return (-1,0)
    
    # we weight on total time these two CPUs spent on this task
    mer['totTime'] = mer['nevents_f','sum']*mer['cpuPerEvent_f','mean'] + mer['nevents_s','sum']*mer['cpuPerEvent_s','mean']
    tot=mer.totTime.sum()
    mer['weight'] = mer['totTime']/tot
    
    # print(mer)
    
    sc1 = (mer['cpuPerEvent_f','mean'] * mer['weight']).mean()
    sc2 = (mer['cpuPerEvent_s','mean'] * mer['weight']).mean()
    if sc1/sc2>10 or sc1/sc2<0.1: 
        print('hard to belive- cpu1:', c1, "cpu2:",c2,'\nratio:',sc1/sc2  )
        return (-1,0)
    return (sc1/sc2, mer.shape[0])

#sumup('Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz HT','Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz HT')

#### double loop over CPUs (half loop) for each combination go through raw data and sum up cputimes weighted by number of events 

In [10]:
c2c = pd.DataFrame(0.0, index=unique_cpus, columns=unique_cpus)
# c2c.head()

for i,c1 in enumerate(unique_cpus):
    print(i, end=" ")
    for j,c2 in enumerate(unique_cpus):
        if j<=i: continue
        (res,siz) = sumup(c1,c2)
        #print(i, j, res, ' based on ',siz, 'tasks')
        c2c.set_value(c1,c2,res)
c2c.head()
c2c.to_csv('c2c.csv')

0 1 2 3 4 hard to belive- cpu1: AMD Opteron(TM) Processor 6274 HT cpu2: Intel(R) Xeon(R) CPU E5-2640 v3 @ 2.60GHz VM 
ratio: 13.250698273462907
5 6 7 8 hard to belive- cpu1: AMD Opteron(tm) Processor 6168 cpu2: Intel(R) Xeon(R) CPU E5-2640 v3 @ 2.60GHz VM 
ratio: 13.186793190012358
9 10 11 12 13 14 15 16 17 hard to belive- cpu1: Intel(R) Xeon(R) CPU           E5345  @ 2.33GHz cpu2: Intel(R) Xeon(R) CPU           L5640  @ 2.27GHz VM 
ratio: 105.30379962450607
hard to belive- cpu1: Intel(R) Xeon(R) CPU           E5345  @ 2.33GHz cpu2: Intel(R) Xeon(R) CPU E5-2640 v3 @ 2.60GHz VM 
ratio: 328.67420479547366
18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 hard to belive- cpu1: Intel(R) Xeon(R) CPU           L5640  @ 2.27GHz VM cpu2: Intel(R) Xeon(R) CPU           X5355  @ 2.66GHz 
ratio: 0.0028869727866065514
hard to belive- cpu1: Intel(R) Xeon(R) CPU           L5640  @ 2.27GHz VM cpu2: Intel(R) Xeon(R) CPU           X5570  @ 2.93GHz 
ratio: 0.008143310191797221
hard to

#### helpful functions

In [11]:
def cpuVScpu(x1=[],y1=[],x2=[],y2=[],x3=[],y3=[],xt="x",yt="y", tit="", fit=True):
#     print ("test:",x1,y1)
#     print ("test_prediction:",x2,y2)
#     print ("Train:", x3.flatten().tolist(),y3.flatten().tolist())
    fig = plt.figure()
    fig.set_size_inches(10,10)
    ax1 = fig.add_subplot(111)
    sec = ax1.scatter(x3.flatten().tolist(), y3.flatten().tolist(), label='train',c='b')
    scp = ax1.scatter(x1, y1, label='test', c='g')
    scp = ax1.scatter(x2, y2, label='test_prediction', marker="s",c='r')
    plt.xlabel(xt)
    plt.ylabel(yt)
    plt.title(tit)
    plt.legend()
    if fit:
        lm = LReg.fit(x3, y3)
        print ("Linear model:",lm.coef_ , lm.intercept_)
        
        x_fit = np.linspace(x3[0], x3[-1], 2)
        y_fit = x_fit * lm.coef_[0][0] + lm.intercept_
        
        ax1.plot(x_fit, y_fit,'-', color='red')
        